# Distances measurement stations along netwerk

In [3]:
library(tidyverse)

## Normalize and combine all measurement stations

### Receivers

In [3]:
eels <- read_csv("..//files//eels.csv", col_types = cols())

In [4]:
receivers <- eels %>% 
    select(Receiver, latitude, longitude, station) %>% 
    distinct() %>%
    mutate(station_type = "receiver") %>%
    rename(station_id = Receiver,
           station_name = station)

In [5]:
head(receivers)

station_id,latitude,longitude,station_name,station_type
VR2W-112287,51.00588,3.77876,s-2,receiver
VR2W-112297,51.00164,3.85695,s-Wetteren,receiver
VR2W-115438,51.01680,3.92527,s-Wichelen,receiver
VR2W-122325,51.04340,4.04783,s-3,receiver
VR2W-122322,51.02032,3.96965,s-2a,receiver
VR2W-112292,51.03855,4.11750,s-Dendermonde,receiver


### Tidal stations

Using the [tidal stations overview](https://github.com/inbo/wateRinfo/files/1483458/tij_all_identifiers.txt) as discussed on the wateRinfo [GitHub Issues](https://github.com/inbo/wateRinfo/issues/11#issuecomment-345339779):

In [8]:
ts_id_tide <- read_csv("../files/scheldt_tide_waterinfo_identifiers.csv", col_types = cols()) 

In [11]:
ts_id_tide %>%
    filter(grepl(".*10min.base", ts_name)) %>%
    filter(!station_name %in% c("Walem tij/Rupel", "Duffel-sluis tij", "Lier Molbrug tij/Nete",
                           "Kessel tij/Grote Nete", "Emblem tij/Kleine Nete", 
                           "Mechelen Benedensluis tij/Dijle", "Mechelen Stuw Opwaarts tij/Dijle",
                           "Hombeek tij/Zenne", "Zemst tij/Zenne", "Gentbrugge tij/Zeeschelde",
                           "Waasmunster Manta tij/Durme")) %>% # exclude nonte-scheldt stations
    select(station_no, station_name, 
           station_longitude, station_latitude) %>%
    distinct() %>%
    mutate(station_type = "waterinfo") %>%
    rename(station_id = station_no,
           longitude = station_longitude,
           latitude = station_latitude)

station_id,station_name,longitude,latitude,station_type
dur01a-1066,Tielrode tij/Durme,4.174059,51.10959,waterinfo
zes01a-1066,Prosperpolder tij/Zeeschelde,4.237937,51.34833,waterinfo
zes10a-1066,Liefkenshoek tij/Zeeschelde,4.286840,51.29659,waterinfo
zes14a-1066,Kallosluis tij/Zeeschelde,4.296636,51.26544,waterinfo
zes21a-1066,Antwerpen tij/Zeeschelde,4.397031,51.22882,waterinfo
zes28a-1066,Hemiksem tij/Zeeschelde,4.328803,51.14298,waterinfo
zes36a-1066,Temse tij/Zeeschelde,4.218673,51.12281,waterinfo
zes42a-1066,Sint-Amands tij/Zeeschelde,4.197214,51.05264,waterinfo
zes47a-1066,Dendermonde tij/Zeeschelde,4.103856,51.03456,waterinfo
zes49a-1066,Schoonaarde tij/Zeeschelde,4.007586,51.00467,waterinfo


### Combine receivers and waterinfo stations

In [9]:
stations <- bind_rows(tidal_stations, receivers)

In [55]:
stations %>% filter(station_name == "s-4c")

station_id,station_name,longitude,latitude,station_type
VR2W-122336,s-4c,4.19007,51.07363,receiver
VR2W-122339,s-4c,4.19007,51.07363,receiver


## Derive internal distances stations

In [12]:
library(sp)
library(raster)

Load the study area rivers/canals as a binary mask

In [13]:
study_area <- raster("../files/study_area_binary.tif")

Convert the CRS of the stations to the CRS of the binary mask

In [14]:
crs_mask <- CRS("+init=epsg:32631") # use gdalinfo to derive the CRS
crs_stations <- CRS("+init=epsg:4326")

In [15]:
# project coordinates
stations_sp <- SpatialPointsDataFrame(coords = stations[, c("longitude", 
                                                            "latitude")],
                                      data = stations,
                                      proj4string = crs_stations)
stations_sp <- spTransform(stations_sp, crs_mask)

Load the support functions to derive internal distances

In [17]:
source('https://raw.githubusercontent.com/inbo/fish-tracking/master/scripts/receiver_distance_analysis/receiver_distance_fun.R?token=AAuErpSpvzyAyQUBCN4zLfan07fkHZ5mks5bMj8XwA%3D%3D')

Update the binary mask for stations that fall outside the binary mask. The basic strategy is to extend the points raster point size until it overlaps with the main binary mask:

In [18]:
study_area_binary_extended <- adapt.binarymask(study_area, stations_sp)

[1] 4
[1] 4
[1] 4
[1] 4
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 3
[1] 2
[1] 2
[1] 2
[1] 2
[1] 2
[1] 2
[1] 2
NULL


In [34]:
# check on memory usage of R objects in memory
#sort( sapply(ls(), function(x){object.size(get(x))})) 

Check if the extended mask is properly derived to extract in-network distances:

In [35]:
control.mask(study_area_binary_extended, stations_sp)

[1] TRUE

Calculate all internal distances for the stations:

In [36]:
cst_dst_frame <- get.distance.matrix(study_area_binary_extended, stations_sp)

Write the internal distances to file

In [69]:
cst_dst_frame <- as.data.frame(cst_dst_frame)

In [58]:
write_csv(cst_dst_frame, "../files/stations_internal_distances.csv")

## Translate to distance from sea (i.e. reference station close to sea)

When you do not want to reproduce the entire calculation

Fix on station s-4c **TOFIX!**

In [77]:
cst_dst_frame <- cst_dst_frame[, -33]

In [95]:
cst_dst_frame %>%
    rownames_to_column("STATION") %>%
    filter(STATION == "ws-DL7") %>%
    gather(key = "station", value = "distance_to_refence_station", -STATION) %>%
    rename(reference_station = STATION) %>%
    arrange(distance_to_refence_station) %>%
    inner_join(stations, by = c("station" = "station_name")) -> distance_from_reference

In [96]:
write_csv(distance_from_reference, "../files/distance_from_sea.csv")